# Assignment 3


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
from datetime import datetime
import numpy as np

In [2]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'
# url = 'COVID-19.csv'
df = pd.read_csv(url)


In [3]:
# Ensuring the Date is just date, and not datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

,X,Y,Date,ConfirmedCovidCases,TotalConfirmedCovidCases,ConfirmedCovidDeaths,TotalCovidDeaths,StatisticsProfileDate,CovidCasesConfirmed,HospitalisedCovidCases,...,CommunityTransmission,CloseContact,TravelAbroad,FID,HospitalisedAged65to74,HospitalisedAged75to84,HospitalisedAged85up,Aged65to74,Aged75to84,Aged85up
0,-7.692596,53.288234,2020-02-29,1,1,0.0,0,2020/02/27 00:00:00+00,NaN,NaN,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,-7.692596,53.288234,2020-03-03,1,2,0.0,0,2020/03/01 00:00:00+00,NaN,NaN,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN
2,-7.692596,53.288234,2020-03-04,4,6,0.0,0,2020/03/02 00:00:00+00,1.0,0.0,...,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
3,-7.692596,53.288234,2020-03-05,7,13,0.0,0,2020/03/03 00:00:00+00,2.0,1.0,...,0,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
4,-7.692596,53.288234,2020-03-06,5,18,0.0,0,2020/03/04 00:00:00+00,5.0,4.0,...,0,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# examining columns and rows
df.shape

(648, 41)

In [5]:
# checking type for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   X                            648 non-null    float64
 1   Y                            648 non-null    float64
 2   Date                         648 non-null    object 
 3   ConfirmedCovidCases          648 non-null    int64  
 4   TotalConfirmedCovidCases     648 non-null    int64  
 5   ConfirmedCovidDeaths         499 non-null    float64
 6   TotalCovidDeaths             648 non-null    int64  
 7   StatisticsProfileDate        648 non-null    object 
 8   CovidCasesConfirmed          646 non-null    float64
 9   HospitalisedCovidCases       646 non-null    float64
 10  RequiringICUCovidCases       646 non-null    float64
 11  HealthcareWorkersCovidCases  646 non-null    float64
 12  ClustersNotified             626 non-null    float64
 13  HospitalisedAged5   

In [6]:
# examining the names of the columns
df.columns

Index(['X', 'Y', 'Date', 'ConfirmedCovidCases', 'TotalConfirmedCovidCases',
       'ConfirmedCovidDeaths', 'TotalCovidDeaths', 'StatisticsProfileDate',
       'CovidCasesConfirmed', 'HospitalisedCovidCases',
       'RequiringICUCovidCases', 'HealthcareWorkersCovidCases',
       'ClustersNotified', 'HospitalisedAged5', 'HospitalisedAged5to14',
       'HospitalisedAged15to24', 'HospitalisedAged25to34',
       'HospitalisedAged35to44', 'HospitalisedAged45to54',
       'HospitalisedAged55to64', 'Male', 'Female', 'Unknown', 'Aged1to4',
       'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44', 'Aged45to54',
       'Aged55to64', 'Median_Age', 'CommunityTransmission', 'CloseContact',
       'TravelAbroad', 'FID', 'HospitalisedAged65to74',
       'HospitalisedAged75to84', 'HospitalisedAged85up', 'Aged65to74',
       'Aged75to84', 'Aged85up'],
      dtype='object')

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [7]:
df['X'].unique()

array([-7.692596])

In [8]:
df['Y'].unique()

array([53.288234])

In [9]:
df['CommunityTransmission'].unique()

array([0], dtype=int64)

In [10]:
df['CloseContact'].unique()

array([0], dtype=int64)

In [11]:
df['TravelAbroad'].unique()

array([0], dtype=int64)

### Dropping Columns (X and Y)

| Column | Reason |
|---|---|
| df['X'] | no unique values |
| df['Y'] | no unique values |
| df['StatisticsProfileDate'] | Very similar to df['Date']|
| df['CovidCasesConfirmed'] | Very similar to df['Total Confirmed Covid Cases'] |
| df['FID'] | very similar to Index in pandas |
| df['CommunityTransmission'] | no unique values |
| df['CloseContact'] | no unique values |
| df['TravelAbroad'] | no unique values |


In [12]:
# Dropping irrelevant columns
df = df.drop(columns = ['X','Y','CovidCasesConfirmed','StatisticsProfileDate', 'FID', 
                        'CommunityTransmission', 'CloseContact','TravelAbroad'])

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [13]:
df['Date'].unique()

array([datetime.date(2020, 2, 29), datetime.date(2020, 3, 3),
       datetime.date(2020, 3, 4), datetime.date(2020, 3, 5),
       datetime.date(2020, 3, 6), datetime.date(2020, 3, 7),
       datetime.date(2020, 3, 8), datetime.date(2020, 3, 9),
       datetime.date(2020, 3, 10), datetime.date(2020, 3, 11),
       datetime.date(2020, 3, 12), datetime.date(2020, 3, 13),
       datetime.date(2020, 3, 14), datetime.date(2020, 3, 15),
       datetime.date(2020, 3, 16), datetime.date(2020, 3, 17),
       datetime.date(2020, 3, 18), datetime.date(2020, 3, 19),
       datetime.date(2020, 3, 20), datetime.date(2020, 3, 21),
       datetime.date(2020, 3, 22), datetime.date(2020, 3, 23),
       datetime.date(2020, 3, 24), datetime.date(2020, 3, 25),
       datetime.date(2020, 3, 26), datetime.date(2020, 3, 27),
       datetime.date(2020, 3, 28), datetime.date(2020, 3, 29),
       datetime.date(2020, 3, 30), datetime.date(2020, 3, 31),
       datetime.date(2020, 4, 1), datetime.date(2020, 4, 2),
 

## Melting 

In [14]:
# Melting
id_vars = ["Date"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="Column", value_name="Cases")

df.head(10)


,Date,Column,Cases
0,2020-02-29,ConfirmedCovidCases,1.0
1,2020-03-03,ConfirmedCovidCases,1.0
2,2020-03-04,ConfirmedCovidCases,4.0
3,2020-03-05,ConfirmedCovidCases,7.0
4,2020-03-06,ConfirmedCovidCases,5.0
5,2020-03-07,ConfirmedCovidCases,1.0
6,2020-03-08,ConfirmedCovidCases,2.0
7,2020-03-09,ConfirmedCovidCases,3.0
8,2020-03-10,ConfirmedCovidCases,10.0
9,2020-03-11,ConfirmedCovidCases,9.0


# Getting the final dataset

In [15]:
df.head()

,Date,Column,Cases
0,2020-02-29,ConfirmedCovidCases,1.0
1,2020-03-03,ConfirmedCovidCases,1.0
2,2020-03-04,ConfirmedCovidCases,4.0
3,2020-03-05,ConfirmedCovidCases,7.0
4,2020-03-06,ConfirmedCovidCases,5.0


In [16]:
df['Age Range'] = df['Column']
df.tail()

,Date,Column,Cases,Age Range
20731,2021-12-05,Aged85up,9786.0,Aged85up
20732,2021-12-06,Aged85up,9797.0,Aged85up
20733,2021-12-07,Aged85up,9808.0,Aged85up
20734,2021-12-08,Aged85up,9826.0,Aged85up
20735,2021-12-09,Aged85up,9834.0,Aged85up


In [17]:
df['Age Range'] = df['Age Range'].str.replace('to','-')

In [18]:
df['Age Range']= df['Age Range'].str.replace('up','+')

In [19]:
df['Age Range']

0        ConfirmedCovidCases
1        ConfirmedCovidCases
2        ConfirmedCovidCases
3        ConfirmedCovidCases
4        ConfirmedCovidCases
                ...         
20731                Aged85+
20732                Aged85+
20733                Aged85+
20734                Aged85+
20735                Aged85+
Name: Age Range, Length: 20736, dtype: object

In [20]:

df['Age Range']= df['Age Range'].replace("TotalConfirmedCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("ConfirmedCovidCases",np.nan)
df['Age Range']= df['Age Range'].replace("TotalCovidDeaths", np.nan)
df['Age Range']= df['Age Range'].replace("ConfirmedCovidDeaths",np.nan)
df['Age Range']= df['Age Range'].replace("HospitalisedCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("RequiringICUCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("HealthcareWorkersCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("ClustersNotified", np.nan)
df['Age Range']= df['Age Range'].str.replace("Aged", '')
df['Age Range']= df['Age Range'].str.replace("Hospitalised5", '1-4')
df['Age Range']= df['Age Range'].str.replace("1-4-14",'5-14')
df['Age Range']= df['Age Range'].str.replace("1-45-64",'55-64')
df['Age Range']= df['Age Range'].str.replace("Hospitalised", '')
df['Age Range']= df['Age Range'].replace("Median_Age", np.nan)
df['Age Range']= df['Age Range'].replace("Male", np.nan)
df['Age Range']= df['Age Range'].replace("Female", np.nan)
df['Age Range']= df['Age Range'].replace("Female", np.nan)
df['Age Range']= df['Age Range'].replace("Unknown",np.nan)

In [21]:
df["Age Range"].unique()

array([nan, '1-4', '5-14', '15-24', '25-34', '35-44', '45-54', '55-64',
       '65-74', '75-84', '85+'], dtype=object)

In [22]:
df["Column"].unique()

array(['ConfirmedCovidCases', 'TotalConfirmedCovidCases',
       'ConfirmedCovidDeaths', 'TotalCovidDeaths',
       'HospitalisedCovidCases', 'RequiringICUCovidCases',
       'HealthcareWorkersCovidCases', 'ClustersNotified',
       'HospitalisedAged5', 'HospitalisedAged5to14',
       'HospitalisedAged15to24', 'HospitalisedAged25to34',
       'HospitalisedAged35to44', 'HospitalisedAged45to54',
       'HospitalisedAged55to64', 'Male', 'Female', 'Unknown', 'Aged1to4',
       'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44',
       'Aged45to54', 'Aged55to64', 'Median_Age', 'HospitalisedAged65to74',
       'HospitalisedAged75to84', 'HospitalisedAged85up', 'Aged65to74',
       'Aged75to84', 'Aged85up'], dtype=object)

In [23]:
df = df [[
    "Date",
    "Column",
    "Age Range",
    "Cases"
]]

df.head()

,Date,Column,Age Range,Cases
0,2020-02-29,ConfirmedCovidCases,NaN,1.0
1,2020-03-03,ConfirmedCovidCases,NaN,1.0
2,2020-03-04,ConfirmedCovidCases,NaN,4.0
3,2020-03-05,ConfirmedCovidCases,NaN,7.0
4,2020-03-06,ConfirmedCovidCases,NaN,5.0


In [24]:
#        'HospitalisedAged5', 'HospitalisedAged5to14',
#        'HospitalisedAged15to24', 'HospitalisedAged25to34',
#        'HospitalisedAged35to44', 'HospitalisedAged45to54',
#        'HospitalisedAged55to64', 
#        'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44',
#        'Aged45to54', 'Aged55to64',


# checking that 'HospitalisedAged5' with have 5
df[df["Column"] =='HospitalisedAged55to64'].head()

,Date,Column,Age Range,Cases
9072,2020-02-29,HospitalisedAged55to64,55-64,NaN
9073,2020-03-03,HospitalisedAged55to64,55-64,NaN
9074,2020-03-04,HospitalisedAged55to64,55-64,0.0
9075,2020-03-05,HospitalisedAged55to64,55-64,0.0
9076,2020-03-06,HospitalisedAged55to64,55-64,0.0


In [25]:
# checking that 'HospitalisedAged85up' with have 85
df[df["Column"] =='HospitalisedAged85up'].head()

,Date,Column,Age Range,Cases
18144,2020-02-29,HospitalisedAged85up,85+,NaN
18145,2020-03-03,HospitalisedAged85up,85+,NaN
18146,2020-03-04,HospitalisedAged85up,85+,0.0
18147,2020-03-05,HospitalisedAged85up,85+,0.0
18148,2020-03-06,HospitalisedAged85up,85+,0.0


### Dropping rows with NaN in cases

In [26]:
df.head(20)

,Date,Column,Age Range,Cases
0,2020-02-29,ConfirmedCovidCases,NaN,1.0
1,2020-03-03,ConfirmedCovidCases,NaN,1.0
2,2020-03-04,ConfirmedCovidCases,NaN,4.0
3,2020-03-05,ConfirmedCovidCases,NaN,7.0
4,2020-03-06,ConfirmedCovidCases,NaN,5.0
5,2020-03-07,ConfirmedCovidCases,NaN,1.0
6,2020-03-08,ConfirmedCovidCases,NaN,2.0
7,2020-03-09,ConfirmedCovidCases,NaN,3.0
8,2020-03-10,ConfirmedCovidCases,NaN,10.0
9,2020-03-11,ConfirmedCovidCases,NaN,9.0


In [27]:
# df = df[~df['Cases'].isna()]
# df["Cases"] = df["Cases"].astype(int)
# reseting the dataframe index
df = df.reset_index(drop = True)

In [28]:
df.head()

,Date,Column,Age Range,Cases
0,2020-02-29,ConfirmedCovidCases,NaN,1.0
1,2020-03-03,ConfirmedCovidCases,NaN,1.0
2,2020-03-04,ConfirmedCovidCases,NaN,4.0
3,2020-03-05,ConfirmedCovidCases,NaN,7.0
4,2020-03-06,ConfirmedCovidCases,NaN,5.0


In [29]:
df.shape

(20736, 4)

### Dealing with ['Column'] column

In [30]:
# Testing the regular expression
line = 'HospitalisedAged45to54'
match = re.findall('([A-Z][a-z]+)', line)
new_line = match[0] + " " + match[1] 
print(new_line)

Hospitalised Aged


In [31]:
column_list = []
for i in  df["Column"]:
    match = re.findall('([A-Z][a-z]+)', i)
    if len(match) == 4:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2] + ' ' + match[3]
    elif len(match) == 3:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2]
    elif len(match) == 2:
        new_line = match[0] + ' ' + match[1] 
    else:
        new_line = match[0]

    column_list.append(new_line)

# examining the list
print(column_list)  

['Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases', 'Confirmed Covid Cases',

In [32]:
column_list = pd.Series(column_list)
column_list = column_list.reset_index()

In [33]:
df = pd.merge(df, column_list, left_index = True, right_index = True)

In [34]:
df = df.rename(columns={'Column': "old_column", 0 : "Column"})

In [35]:
df = df[[
    "Date",
    "old_column",
    "Column",
    "Age Range",
    "Cases"
]]

In [36]:
df.head()

,Date,old_column,Column,Age Range,Cases
0,2020-02-29,ConfirmedCovidCases,Confirmed Covid Cases,NaN,1.0
1,2020-03-03,ConfirmedCovidCases,Confirmed Covid Cases,NaN,1.0
2,2020-03-04,ConfirmedCovidCases,Confirmed Covid Cases,NaN,4.0
3,2020-03-05,ConfirmedCovidCases,Confirmed Covid Cases,NaN,7.0
4,2020-03-06,ConfirmedCovidCases,Confirmed Covid Cases,NaN,5.0


### Daily Cases

In [37]:
hospitalised_df = df[df['Column'].str.contains("Hospitalised Aged")]
aged_df = df[df['Column'].str.startswith("Aged")]

In [38]:
age_range_65up = hospitalised_df[hospitalised_df['Age Range'] == '65-74']
age_range_65up.tail(10)

,Date,old_column,Column,Age Range,Cases
17486,2021-11-30,HospitalisedAged65to74,Hospitalised Aged,65-74,3521.0
17487,2021-12-01,HospitalisedAged65to74,Hospitalised Aged,65-74,3528.0
17488,2021-12-02,HospitalisedAged65to74,Hospitalised Aged,65-74,3532.0
17489,2021-12-03,HospitalisedAged65to74,Hospitalised Aged,65-74,3540.0
17490,2021-12-04,HospitalisedAged65to74,Hospitalised Aged,65-74,3550.0
17491,2021-12-05,HospitalisedAged65to74,Hospitalised Aged,65-74,3559.0
17492,2021-12-06,HospitalisedAged65to74,Hospitalised Aged,65-74,3560.0
17493,2021-12-07,HospitalisedAged65to74,Hospitalised Aged,65-74,3562.0
17494,2021-12-08,HospitalisedAged65to74,Hospitalised Aged,65-74,3569.0
17495,2021-12-09,HospitalisedAged65to74,Hospitalised Aged,65-74,3573.0


In [39]:
age_list = df['Age Range'].unique()

# removing NaN from the dataset
age_list = age_list[1:]
print(age_list)

['1-4' '5-14' '15-24' '25-34' '35-44' '45-54' '55-64' '65-74' '75-84'
 '85+']


In [40]:
aged_df.head()

,Date,old_column,Column,Age Range,Cases
11664,2020-02-29,Aged1to4,Aged,1-4,NaN
11665,2020-03-03,Aged1to4,Aged,1-4,NaN
11666,2020-03-04,Aged1to4,Aged,1-4,0.0
11667,2020-03-05,Aged1to4,Aged,1-4,0.0
11668,2020-03-06,Aged1to4,Aged,1-4,0.0


### Columns that are Not Accumulated
'Confirmed Covid Cases', 'Total Confirmed Covid Cases','Confirmed Covid Deaths', 'Total Covid Deaths',

In [41]:
df['Column'].unique()

array(['Confirmed Covid Cases', 'Total Confirmed Covid Cases',
       'Confirmed Covid Deaths', 'Total Covid Deaths',
       'Hospitalised Covid Cases', 'Requiring Covid Cases',
       'Healthcare Workers Covid Cases', 'Clusters Notified',
       'Hospitalised Aged', 'Male', 'Female', 'Unknown', 'Aged',
       'Median Age'], dtype=object)

In [42]:
column_list = ['Confirmed Covid Cases', 'Total Confirmed Covid Cases',
       'Confirmed Covid Deaths', 'Total Covid Deaths',
       'Hospitalised Covid Cases', 'Requiring Covid Cases',
       'Healthcare Workers Covid Cases', 'Clusters Notified',
       'Hospitalised Aged', 'Male', 'Female', 'Unknown', 'Aged',
       'Median Age']
column_list

['Confirmed Covid Cases',
 'Total Confirmed Covid Cases',
 'Confirmed Covid Deaths',
 'Total Covid Deaths',
 'Hospitalised Covid Cases',
 'Requiring Covid Cases',
 'Healthcare Workers Covid Cases',
 'Clusters Notified',
 'Hospitalised Aged',
 'Male',
 'Female',
 'Unknown',
 'Aged',
 'Median Age']

In [43]:
# Columns that have accumulated cases and no daily cases count 
accumulated = ['Male','Hospitalised Covid Cases', 
               'Healthcare Workers Covid Cases',  'Unknown', 'Female',
               'Median Age', 'Requiring Covid Cases', 'Clusters Notified']

In [44]:
df_list = []


for i in column_list:
    if "Hospitalised Aged" == i: 
        for age in age_list:
            h_df = hospitalised_df[hospitalised_df['Age Range'].str.contains(age)]
            h_df = pd.DataFrame(h_df)
            h_df['Daily Cases'] = h_df['Cases'].diff(1)
            h_df['Diff'] = h_df['Daily Cases'].diff(1)
#             h_df = h_df.dropna()
#             h_df["Daily Cases"] = h_df["Daily Cases"].astype(int)
#             h_df["Diff"] = h_df["Diff"].astype(int)
            df_list.append(h_df)
        
    elif "Aged" == i: 
        for age in age_list:
            a_df = aged_df[aged_df['Age Range'].str.contains(age)]
            a_df = pd.DataFrame(a_df)
            a_df['Daily Cases'] = a_df['Cases'].diff(1)
            a_df['Diff'] = a_df['Daily Cases'].diff(1)
#             a_df = a_df.dropna()
#             a_df["Daily Cases"] = a_df["Daily Cases"].astype(int)
#             a_df["Diff"] = a_df["Diff"].astype(int)
            df_list.append(a_df)
    
    elif  i in accumulated:
        n_df = df[df['Column'].str.contains(i)]
        n_df = pd.DataFrame(n_df)
        n_df['Daily Cases'] = n_df['Cases'].diff(1)
        n_df['Diff'] = n_df['Daily Cases'].diff(1)
#         n_df = n_df.dropna()
#         n_df["Daily Cases"] = n_df["Daily Cases"].astype(int)
#         n_df["Diff"] = n_df["Diff"].astype(int)
        df_list.append(n_df)
    else:
        n_df = df[df['Column'].str.contains(i)]
        n_df = pd.DataFrame(n_df)
        n_df['Daily Cases'] = np.nan
        n_df['Diff'] = n_df['Cases'].diff(1)
#         n_df = n_df.dropna()
#         n_df["Daily Cases"] = n_df["Daily Cases"].astype(int)
#         n_df["Diff"] = n_df["Diff"].astype(int)
        df_list.append(n_df) 

In [45]:
df =  pd.concat(df_list)

In [46]:
# sorting data by date
df = df.sort_values(ascending = True, by = ["Date"])

# reseting the dataframe index
df = df.reset_index(drop=True)

In [47]:
df.head(50)

,Date,old_column,Column,Age Range,Cases,Daily Cases,Diff
0,2020-02-29,ConfirmedCovidCases,Confirmed Covid Cases,NaN,1.0,NaN,NaN
1,2020-02-29,TotalConfirmedCovidCases,Total Confirmed Covid Cases,NaN,1.0,NaN,NaN
2,2020-02-29,Aged85up,Aged,85+,NaN,NaN,NaN
3,2020-02-29,HospitalisedAged25to34,Hospitalised Aged,25-34,NaN,NaN,NaN
4,2020-02-29,Unknown,Unknown,NaN,NaN,NaN,NaN
5,2020-02-29,HospitalisedAged15to24,Hospitalised Aged,15-24,NaN,NaN,NaN
6,2020-02-29,Aged1to4,Aged,1-4,NaN,NaN,NaN
7,2020-02-29,Aged55to64,Aged,55-64,NaN,NaN,NaN
8,2020-02-29,HospitalisedAged85up,Hospitalised Aged,85+,NaN,NaN,NaN
9,2020-02-29,HospitalisedAged55to64,Hospitalised Aged,55-64,NaN,NaN,NaN


In [48]:
df.tail(50)

,Date,old_column,Column,Age Range,Cases,Daily Cases,Diff
21334,2021-12-08,HospitalisedAged5,Hospitalised Aged,1-4,424.0,2.0,0.0
21335,2021-12-08,HospitalisedAged85up,Hospitalised Aged,85+,2618.0,6.0,6.0
21336,2021-12-08,HospitalisedAged35to44,Hospitalised Aged,35-44,1920.0,7.0,4.0
21337,2021-12-08,HospitalisedAged5to14,Hospitalised Aged,5-14,311.0,4.0,4.0
21338,2021-12-08,Aged45to54,Aged,45-54,77591.0,787.0,416.0
21339,2021-12-08,Aged85up,Aged,85+,9826.0,18.0,7.0
21340,2021-12-08,Median_Age,Median Age,NaN,32.0,1.0,2.0
21341,2021-12-08,HospitalisedAged65to74,Hospitalised Aged,65-74,3569.0,7.0,5.0
21342,2021-12-08,Aged75to84,Aged,75-84,16401.0,45.0,25.0
21343,2021-12-08,TotalConfirmedCovidCases,Total Confirmed Covid Cases,NaN,606852.0,NaN,4126.0


In [50]:
# dropping old_column
df = df.drop(columns = ['old_column'])

In [52]:
df[df['Column'] == 'Aged']

,Date,Column,Age Range,Cases,Daily Cases,Diff
2,2020-02-29,Aged,85+,NaN,NaN,NaN
6,2020-02-29,Aged,1-4,NaN,NaN,NaN
7,2020-02-29,Aged,55-64,NaN,NaN,NaN
11,2020-02-29,Aged,5-14,NaN,NaN,NaN
14,2020-02-29,Aged,45-54,NaN,NaN,NaN
...,...,...,...,...,...,...
21371,2021-12-09,Aged,45-54,78143.0,552.0,-235.0
21375,2021-12-09,Aged,25-34,98686.0,639.0,-173.0
21377,2021-12-09,Aged,15-24,110075.0,436.0,-252.0
21381,2021-12-09,Aged,1-4,26257.0,223.0,-75.0


### Saving to a new csv file

In [51]:
df.to_csv(r"tidy_data_df.csv", index =  False)